In [2]:
# Importing LIBRARIES
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import os
import numpy as np
import pandas as np
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [3]:
# Specifying the path of the data(train,test,validaton)
train = 'C:/Users/Kibet/Desktop/General/project/classifier/input/chest_xray/chest_xray/train'
test = 'C:/Users/Kibet/Desktop/General/project/classifier/input/chest_xray/chest_xray/test'
val = 'C:/Users/Kibet/Desktop/General/project/classifier/input/chest_xray/chest_xray/val'

In [4]:
# Here we have backended keras to tensorflow ,so we go for channel_last ie to specify
#the channel value as the last dimension in shape of the input.
img_width,img_height= 150,150
input_shape = (img_width,img_height,3)

In [5]:
#Creating Model
model = Sequential()

# The number of filters are 32 and the kernal_size is (3,3)
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  
model.add(Dropout(0.5)) 
          
model.add(Dense(16, activation='relu'))            
model.add(Dropout(0.5))
    
model.add(Dense(1, activation='sigmoid'))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
# Here we use RMSPROP optimizer and BINARY_CROSSENTROPY as loss function
model.compile(loss='binary_crossentropy',
optimizer='rmsprop',
metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 175232)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 175232)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                2803728   
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total para

In [8]:
# ImageDataGenerator-Generate batches of tensor image data with real-time data augmentation. 
#The data will be looped over (in batches).

train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2
                                   ,horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [9]:
# Here we import images directly from Directory by using flow_from_directory method.
#flow_from_directory() automatically infers the labels from the directory structure of the folders containing images

train_generator = train_datagen.flow_from_directory(
    train,
    target_size=(img_width, img_height),
    batch_size=16,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test,
    target_size=(img_width, img_height),
    batch_size=16,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    val,
    target_size=(img_width, img_height),
    batch_size=16,
    class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [10]:
# We Fit the model here using fit_generator as we are dealing with large datasets

#Improving accuracy using callbacks 
from keras.callbacks import EarlyStopping
early_stopping=EarlyStopping(monitor='val_loss', 
                             min_delta=0.01, 
                             patience=1, 
                             verbose=0, 
                             mode='auto', 
                             baseline=None, 
                             restore_best_weights=True
                            )

training_visualization=model.fit_generator(
    train_generator,
    steps_per_epoch=5217 // 16,
    callbacks=[early_stopping],
    epochs=3,
    validation_data=validation_generator,
    validation_steps=17 // 16)

Epoch 1/3
326/326 [==============================] - 358s 1s/step - loss: 0.7001 - acc: 0.7400 - val_loss: 0.7032 - val_acc: 0.5000
Epoch 2/3
326/326 [==============================] - 335s 1s/step - loss: 0.6153 - acc: 0.7429 - val_loss: 0.7280 - val_acc: 0.5000


In [14]:
scores=model.evaluate(validation_generator, batch_size=16, verbose=0)

AttributeError: 'DirectoryIterator' object has no attribute 'ndim'

In [ ]:
print('model val loss:', scores[0])
print('model val accuracy:', scores[1])